# #500 Geotechnical Material Library
<i>Creates geotechical material attributes as defined in the corresponding library spreadsheet</i>
***

In [ ]:
material_library_path = "500 Geotechnical Material Library.xlsx"

In [ ]:
import numpy as np
import pandas as pd
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("A model must be open to create the material attributes")

db = lusas.database()
db_units = db.getModelUnits()

STRESS_DIM = lusas.packDimensionality(0,1,-2,0,0,0)
DENSITY_DIM = lusas.packDimensionality(0,0,-3,1,0,0)

In [ ]:
# Get the unit set for which the materials are defined
units = pd.read_excel(material_library_path).columns[1]
lib_units = lusas.getUnitSet(units)

In [ ]:
# Helper function to convert all material parameters to model units
def convert_material_units(params:dict) -> dict:

    params['E']         = lib_units.convertToUnitSetPackedDim(db_units, params['E'],         STRESS_DIM)
    params['CSngle']    = lib_units.convertToUnitSetPackedDim(db_units, params['CSngle'],    STRESS_DIM)
    params['QmaxSngle'] = lib_units.convertToUnitSetPackedDim(db_units, params['QmaxSngle'], STRESS_DIM)
    params['QminSngle'] = lib_units.convertToUnitSetPackedDim(db_units, params['QminSngle'], STRESS_DIM)
    params['Kf']        = lib_units.convertToUnitSetPackedDim(db_units, params['Kf'],        STRESS_DIM)    
    params['rhoF']      = lib_units.convertToUnitSetPackedDim(db_units, params['rhoF'],      DENSITY_DIM)

    return params


In [ ]:
# Read the soil material definitions from the excel file
df_soils = pd.read_excel(material_library_path, header=1)
# Determine the number of material definitions 
no_materials = len(df_soils.columns) - 3
# Create the materials in the model
if no_materials > 0:

    for i in range(3, no_materials+3):
        # Name and parameters in model units
        soil_name = df_soils.iloc[:,i].name
        params = convert_material_units(dict(zip(df_soils.iloc[:,2], df_soils.iloc[:,i])))
        print(soil_name, params)
        # Create the material
        attr = db.createIsotropicMaterial(soil_name, params['E'], params['nu'], params['rho'])
        # Modified Mohr-Coulomb parameters
        attr.addPlasticModifiedMohrCoulomb(params['type_8'], params['frc'], params['dil'], 0, params['dmpFct'])
        attr.addModifiedMohrCoulombCohesion(0, params['CSngle'])
        attr.addModifiedMohrCoulombTensile(0, params['QmaxSngle'])
        attr.addModifiedMohrCoulombCompressive(0, params['QminSngle'])
        # Two Phase
        fluid_bulk_modulus_definition = "Absolute value"
        two_phase_density_type = "Fully saturated"
        if params['saturation'] == 'Fully':
            attr.addTwoPhaseStandard(0.0, params['Kf'], params['n'], 9.81, params['kx'], params['ky'], params['kz'], 
                                                params['rhoF'], 0.0, fluid_bulk_modulus_definition, two_phase_density_type)
        elif params['saturation'] == 'Partially':
            attr.addConstantWaterContentTwoPhase(0.0, params['Kf'], params['n'], 9.81, params['kx'], params['ky'], params['kz'], 
                                                params['rhoF'], params['Swc'], params['Sws'], params['Kred'], 0.0, 
                                                fluid_bulk_modulus_definition, "saturation", two_phase_density_type)
        # Ko Initialisation
        if not isinstance(params['Ko'], str) and not np.isnan(params['Ko']):
            attr.addKoElasticRow(0.0, params['Ko'])